In [1]:
import sys
quic_path = '/home/quic/QUIC-Projects'
if not quic_path in sys.path:
    sys.path.append(quic_path)

In [2]:
import numpy as np 
from matplotlib import pyplot as plt

In [3]:
from classifiers.quantum.qasvm import NormQSVM
from qiskit.circuit.library import ZZFeatureMap
from classifiers.quantum.ansatz import Circuit1
from qiskit.providers.aer import AerSimulator
from qiskit.utils import QuantumInstance
from qiskit import transpile

In [4]:
def my_function(m, n):
    data = np.random.random(size=(2**m, 2**n))
    label = np.where(np.random.random(2**m)>0, 1, 0)
    feature_map = ZZFeatureMap(feature_dimension=2**n)
    var_qc = Circuit1(m, 1, skip_final_rotation_layer=True)
    backend = AerSimulator()
    quantum_instance = QuantumInstance(backend=backend, shots=2**13)
    qasvm = NormQSVM(data, label, quantum_instance=quantum_instance, var_form=var_qc, feature_map=feature_map, lamda=1.0)
    return transpile(qasvm.second_order_circuit, backend=backend, basis_gates=['rx', 'ry', 'rz', 'cx'], optimization_level=0).depth()

In [5]:
from tqdm.notebook import tqdm
from itertools import product
m = np.arange(1, 13)
n = np.arange(1, 13)
X, Y = np.meshgrid(m, n)
Z = np.empty_like(X)
for i, j in tqdm(list(product(range(len(m)), range(len(n))))):
    Z[j, i] = np.log2(my_function(m[i], n[j]))

  0%|          | 0/144 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_wireframe(X, Y, Z, color='black')
ax.set_xlabel('log(M)')
ax.set_ylabel('log(n)')
ax.set_zlabel('log(depth)')

In [ ]:
from sklearn.linear_model import LinearRegression
XY = np.asarray(list(zip(X.flatten(), Y.flatten())))
ZZ = Z.flatten()
reg = LinearRegression().fit(XY, ZZ, sample_weight=(X.flatten()+Y.flatten()))
print("d = O(2^{:.3f} * M^{:.3f} * n^{:.3f}), R^2={:.3f}".format(reg.intercept_, reg.coef_[0], reg.coef_[1], reg.score(XY, ZZ)))

In [9]:
%time my_function(7, 7)

CPU times: user 26min 6s, sys: 9.38 s, total: 26min 16s
Wall time: 26min 17s


4389460

In [ ]:
%time my_function(18, 2)